In [1]:
import torch
import math as ma

def generate_dataset(N = 1000):
    #Fait un tensor N*2 (ensemble de coordonnées (x,y))
    inp = torch.empty(N, 2).uniform_(0, 1) 
    #centre du cercle en 0.5
    a = torch.subtract(inp, 0.5)
    #équation de cercle
    clas = a.pow(2).sum(1).sub(1 / (2*ma.pi)).sign().div(-1).add(1).div(2).long()
    return inp, clas

In [149]:
'''
def LossMSE(input, weights, bias, target):
    N = np.size(target)
    L = torch.sum(torch.pow(output - target,2))/N
    dL_dw = 2 * torch.mul(input, torch.mul(f_prim(input,weights,bias) , (f(input,weights,bias) - target))) /N
    dL_dB = 2 * torch.mul(f_prim(input,weights,bias) , (f(input,weights,bias) - target)) /N
    return L , dL_dw , dL_dB
'''
#some activation functions
######################################################################

def tanh(x):
    y = torch.tanh(x).add(1).div(2)
    return y

def dtanh(x):
    y = (1 - torch.pow(sigma(x),2)).div(2)
    return y
######################################################################

def relu(x):
    a = x.sign().add(1).div(2).long()
    return torch.mul(a,x)

def drelu(x):
    return x.sign().add(1).div(2).long()
######################################################################

#Loss
def loss(v, t):
    y = torch.sum(torch.mul((t-v),(t-v)))
    return y

def dloss(v, t):
    y = torch.mul((t-v),-2)
    return y

#######################################################################
def forward_pass(x, w, b):
    return w.matmul(x) + b
           # torch.matmul(w1,x)+b
   # x1 = sigma(s1)
   # s2 = w2.mv(x1) + b2
   # x2 = sigma(s2)

    #return x0, s1, x1, s2, x2

def backward_pass(w1, b1, w2, b2, w3, b3,
                t,
                x, s1, x1, s2, x2, s3, x3,
                dl_dw1, dl_db1, dl_dw2, dl_db2, dl_dw3, dl_db3):
    x0 = x
    dl_dx3 = dloss(x3, t)
    dl_ds3 = dtanh(s3) * dl_dx3
    dl_dx2 = w3.t().mv(dl_ds3)
    dl_ds2 = drelu(s2) * dl_dx2
    dl_dx1 = w2.t().mv(dl_ds2)
    dl_ds1 = drelu(s1) * dl_dx1

    dl_dw3.add_(dl_ds3.view(-1, 1).mm(x2.view(1, -1)))
    dl_db3.add_(dl_ds3)
    dl_dw2.add_(dl_ds2.view(-1, 1).mm(x1.view(1, -1)))
    dl_db2.add_(dl_ds2)
    dl_dw1.add_(dl_ds1.view(-1, 1).mm(x0.view(1, -1)))
    dl_db1.add_(dl_ds1)
##########################
#update parameters

def optimizer(w,b,lr,dl_dw,dl_db):
    return w - lr * dl_dw, b - lr * dl_db
   
    

In [161]:
### je fais un brouyons pour voir la structure global
#from data.py import generate_dataset
#from modules.py import relu, drelu, sigma, dsigma, forward_pass, backward_pass, loss, dloss, optimizer
#upload les datas
train_input, train_target = generate_dataset(10)
test_input, test_target = generate_dataset(1000)
nb_train_samples = train_input.size(0)
#parametres du network
nb_layers = 3
nb_poids = 25
lr = 0.1/nb_train_samples
#variance des poids a definir
epsilon = 0.0001
error = []
#init 3 hidden layers
w1 = torch.empty(nb_poids, train_input.size(1)).normal_(0, epsilon)
b1 = torch.empty(nb_poids).normal_(0, epsilon)
w2 = torch.empty(nb_poids, nb_poids).normal_(0, epsilon)
b2 = torch.empty(nb_poids).normal_(0, epsilon)
w3 = torch.empty(1,nb_poids).normal_(0, epsilon)
b3 = torch.empty(1).normal_(0, epsilon)

dl_dw1 = torch.empty(w1.size())
dl_db1 = torch.empty(b1.size())
dl_dw2 = torch.empty(w2.size())
dl_db2 = torch.empty(b2.size())
dl_dw3 = torch.empty(w3.size())
dl_db3 = torch.empty(b3.size())

nb_epochs = 50
for epoch in range(nb_epochs):
    er = 0
    dl_dw1.zero_()
    dl_db1.zero_()
    dl_dw2.zero_()
    dl_db2.zero_()
    for i in range(train_input.size()[0]):
    
        s1 = forward_pass(train_input[i], w1, b1)
        x1 = relu(s1)
        s2 = forward_pass(x1, w2, b2)
        x2 = relu(s2)
        s3 = forward_pass(x2, w3, b3)
        x3 = tanh(s3)
        
        if (x3 <= 0.5 and train_target[i] == 1) or (x3 > 0.5 and train_target[i] == 0) :
            er = er + 1
        print(train_target[i],x3)
        backward_pass(w1, b1, w2, b2, w3, b3,
                    train_target[i],
                    train_input[i], s1, x1, s2, x2, s3, x3,
                    dl_dw1, dl_db1, dl_dw2, dl_db2, dl_dw3, dl_db3)
    error.append(er/train_input.size()[0])
    w1, b1 = optimizer(w1,b1,lr,dl_dw1,dl_db1)
    w2, b2 = optimizer(w2,b2,lr,dl_dw2,dl_db2)
    w3, b3 = optimizer(w3,b3,lr,dl_dw3,dl_db3)
        
print(error)
s1 = forward_pass(torch.transpose(test_input, 0, 1), w1, b1[:,None])
x1 = tanh(s1)
s2 = forward_pass(x1, w2, b2[:,None])
x2 = tanh(s2)
s3 = forward_pass(x2, w3, b3[:,None])
x3 = tanh(s3)
y = x3
print(loss(y,test_target))

tensor(0) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(0) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(0) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(0) tensor([0.4999])
tensor(1) tensor([0.4999])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor([1.])
tensor(0) tensor([1.])
tensor(1) tensor(

In [160]:
a = torch.tensor([0.51])

In [21]:
print(a.add(-0.5).sign().add(1).div(2).long())

tensor([1])


In [39]:
train_target

tensor([0, 0, 1, 0, 1])